In [23]:
from story import story

In [24]:
import spacy

nlp = spacy.load("en_core_web_lg")
doc = nlp(" ".join(story))
characters = set()
for tok in doc:
    if tok.pos_ == "PROPN":
        characters.add(tok.text)
characters = list(characters)
availableLora = [
    "<lora:sakayanagiarisu:0.7>, solo_focus, black hat, white bow, school uniform,",
    "<lora:Horikita:0.7> a woman in red school uniform,black hair, red eyes, classroom masterpeace, best quality, (extremely detailed CG:1.4), highly detailed faces",
    "<lora:Amasawa:0.5> a woman in red school uniform,pink - purpule hair, yellow eyes, classroom, tails hairstyle,  an evil grin, slim masterpeace, best quality, (extremely detailed CG:1.4), highly detailed faces ,glowing colors",
    "masterpiece, best quality, 1girl, shadowheart, black hair, braided ponytail, green eyes, circlet, crossed arms, emotionless, looking at viewer, forest, blurry background <lora:ShadowHeart:1>",
]
characterMaps = []
for i in range(len(characters)):
    characterMaps.append(
        {
            "name": characters[i],
            "lora": availableLora[i],
        }
    )
print(characterMaps)

[{'name': 'Sarah', 'lora': '<lora:sakayanagiarisu:0.7>, solo_focus, black hat, white bow, school uniform,'}, {'name': 'Robert', 'lora': '<lora:Horikita:0.7> a woman in red school uniform,black hair, red eyes, classroom masterpeace, best quality, (extremely detailed CG:1.4), highly detailed faces'}, {'name': 'Alice', 'lora': '<lora:Amasawa:0.5> a woman in red school uniform,pink - purpule hair, yellow eyes, classroom, tails hairstyle,  an evil grin, slim masterpeace, best quality, (extremely detailed CG:1.4), highly detailed faces ,glowing colors'}, {'name': 'Shadowheart', 'lora': 'masterpiece, best quality, 1girl, shadowheart, black hair, braided ponytail, green eyes, circlet, crossed arms, emotionless, looking at viewer, forest, blurry background <lora:ShadowHeart:1>'}]


In [25]:
from summarizers import Summarizers
summ = Summarizers()
summarizedStory = []
for chapter in story:
    summarizedStory.append(summ(chapter))

def generate_prompts(sentence):
    noun_phrases = []
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ in ("NOUN", "VERB"):
            noun_phrases.append(token.text)
    return " ".join(noun_phrases)

prompts = []
for sentence in summarizedStory:
    prompt = generate_prompts(sentence)
    prompts.append(prompt)

print(prompts)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['town villain known began terrorizing residents', 'intuition inventor joined forces stop', 'investigated discovered pattern crimes', 'embezzling town funds needed divert attention crimes', 'traps teamwork confronted bringing justice town']


In [26]:
characterCounts = []

def get_characters_in_sentence(sentence, character_names):
  names = []
  for word in sentence.split(" "):
    if word in character_names and word not in names and len(names) < 2:
      names.append(word)

  return names


for sentence in summarizedStory:
    names = get_characters_in_sentence(sentence, characters)
    characterCounts.append(names)

print(characterCounts)


[['Shadowheart'], ['Sarah', 'Alice'], ['Shadowheart'], ['Robert'], ['Sarah', 'Alice']]


In [36]:
import pybase64
import random

def getOpenposeStockImage():
    random_index = random.randint(1, 3)
    with open(f"openpose/{random_index}.png", "rb") as img_file:
        encoded_img = pybase64.b64encode(img_file.read())
    return encoded_img.decode("utf-8")

In [37]:
def getPayloadWithLatentCouple(prompt):
    return {
        "alwayson_scripts": {
            "API payload": {"args": []},
            "Composable Lora": {
                "args": [True, False, False, False, False, False, False]
            },
            "ControlNet": {
                "args": [
                    {
                        "batch_images": "",
                        "control_mode": "Balanced",
                        "enabled": True,
                        "guidance_end": 1,
                        "guidance_start": 0,
                        "image": {"image": getOpenposeStockImage(), "mask": ""},
                        "input_mode": "simple",
                        "is_ui": True,
                        "loopback": False,
                        "low_vram": False,
                        "model": "control_sd15_openpose [fef5e48e]",
                        "module": "openpose",
                        "output_dir": "",
                        "pixel_perfect": False,
                        "processor_res": 512,
                        "resize_mode": "Crop and Resize",
                        "threshold_a": -1,
                        "threshold_b": -1,
                        "weight": 1,
                    },
                    {
                        "batch_images": "",
                        "control_mode": "Balanced",
                        "enabled": False,
                        "guidance_end": 1,
                        "guidance_start": 0,
                        "image": None,
                        "input_mode": "simple",
                        "is_ui": True,
                        "loopback": False,
                        "low_vram": False,
                        "model": "None",
                        "module": "none",
                        "output_dir": "",
                        "pixel_perfect": False,
                        "processor_res": -1,
                        "resize_mode": "Crop and Resize",
                        "threshold_a": -1,
                        "threshold_b": -1,
                        "weight": 1,
                    },
                    {
                        "batch_images": "",
                        "control_mode": "Balanced",
                        "enabled": False,
                        "guidance_end": 1,
                        "guidance_start": 0,
                        "image": None,
                        "input_mode": "simple",
                        "is_ui": True,
                        "loopback": False,
                        "low_vram": False,
                        "model": "None",
                        "module": "none",
                        "output_dir": "",
                        "pixel_perfect": False,
                        "processor_res": -1,
                        "resize_mode": "Crop and Resize",
                        "threshold_a": -1,
                        "threshold_b": -1,
                        "weight": 1,
                    },
                ]
            },
            "Extra options": {"args": []},
            "Latent Couple extension": {
                "args": [
                    True,
                    "1:1,1:2,1:2",
                    "0:0,0:0,0:1",
                    "0.2,0.8,0.8",
                    150,
                    0.2,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                ]
            },
            "OpenPose Editor": {"args": []},
            "Refiner": {"args": [False, "", 0.8]},
            "Seed": {"args": [-1, False, -1, 0, 0, 0]},
        },
        "batch_size": 1,
        "cfg_scale": 7,
        "comments": {},
        "disable_extra_networks": False,
        "do_not_save_grid": False,
        "do_not_save_samples": False,
        "enable_hr": False,
        "height": 512,
        "hr_negative_prompt": "",
        "hr_prompt": "",
        "hr_resize_x": 0,
        "hr_resize_y": 0,
        "hr_scale": 2,
        "hr_second_pass_steps": 0,
        "hr_upscaler": "Latent",
        "n_iter": 1,
        "negative_prompt": "(painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy (painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy, sexy, naked, thighs,",
        "override_settings": {"sd_model_checkpoint": "anythingV3_fp16.ckpt"},
        "override_settings_restore_afterwards": True,
        "prompt": prompt,
        "restore_faces": False,
        "s_churn": 0.0,
        "s_min_uncond": 0.0,
        "s_noise": 1.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "sampler_name": "DPM++ 2M Karras",
        "script_args": [],
        "script_name": None,
        "seed": -1,
        "seed_enable_extras": True,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "steps": 20,
        "styles": [],
        "subseed": -1,
        "subseed_strength": 0,
        "tiling": False,
        "width": 768,
    }


def getPayload(prompt):
    return {
        "prompt": prompt,
        "negative_prompt": "(painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy (painting by bad-artist-anime:0.9), (painting by bad-artist:0.9), watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, artist name, (worst quality, low quality:1.4), bad anatomy, sexy, naked, thighs,",
        "override_settings": {"sd_model_checkpoint": "anythingV3_fp16.ckpt"},
        "override_settings_restore_afterwards": True,
    }

In [38]:
import requests
from PIL import Image, ImageDraw
import base64
import io

payloads = []


def find_by_name(name, list):
    for dict in list:
        if dict["name"] == name:
            return dict


for index, prompt in enumerate(prompts):
    characterCount = characterCounts[index]
    if len(characterCount) == 2:
        for i, character in enumerate(characterCount):
            dict = find_by_name(character, characterMaps)
            prompt += "\nAND\n"
            prompt += dict["lora"]

        payload = getPayloadWithLatentCouple(prompt)
        payloads.append(getPayloadWithLatentCouple(prompt))
    else:
        character = characterCount[0]
        dict = find_by_name(character, characterMaps)
        prompt += " "
        prompt += dict["lora"]
        payloads.append(getPayload(prompt))

for i, payload in enumerate(payloads):
    print(payload["prompt"])
    response = requests.post(
        url=f"http://127.0.0.1:7860/sdapi/v1/txt2img",
        json=payload,
    )
    data = response.json()
    image = Image.open(io.BytesIO(base64.b64decode(data["images"][0])))
    image.save(f"dumps2/sample{i}.png")
    print("image saved")

town villain known began terrorizing residents masterpiece, best quality, 1girl, shadowheart, black hair, braided ponytail, green eyes, circlet, crossed arms, emotionless, looking at viewer, forest, blurry background <lora:ShadowHeart:1>
image saved
intuition inventor joined forces stop
AND
<lora:sakayanagiarisu:0.7>, solo_focus, black hat, white bow, school uniform,
AND
<lora:Amasawa:0.5> a woman in red school uniform,pink - purpule hair, yellow eyes, classroom, tails hairstyle,  an evil grin, slim masterpeace, best quality, (extremely detailed CG:1.4), highly detailed faces ,glowing colors
image saved
investigated discovered pattern crimes masterpiece, best quality, 1girl, shadowheart, black hair, braided ponytail, green eyes, circlet, crossed arms, emotionless, looking at viewer, forest, blurry background <lora:ShadowHeart:1>
image saved
embezzling town funds needed divert attention crimes <lora:Horikita:0.7> a woman in red school uniform,black hair, red eyes, classroom masterpeace,